In [7]:
import numpy as np
import torch as tr
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch

import torch.nn as nn
import torch.nn.functional as F



import torch.optim as optim
import torchvision.transforms as transforms
# from torchvision.datasets import CIFAR10
# from torch.utils.data import DataLoader
# from tqdm.notebook import tqdm as tqdm

import matplotlib.pyplot as plt
import itertools

from train import *
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Here is a dataloader that returns a anchor, positive and negative when iterating over it. This can then be passed into the encoding network and then you can calcualte the loss. This ideally would have a triplet on or triplet off functionality. So, when triplet is off ___get_item___ returns 1 image and corresponding label to be used with classification. 

In [111]:
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor
from torch.utils.data import Dataset, DataLoader

class TripletsMNIST(Dataset):
    def __init__(self, root="./data", train=True, transform=None, triplet_training):
        transform = self.get_default_transform()
        self.mnist_dataset = MNIST(root=root, train=train, transform=transform, download=False)
        self.targets = self.mnist_dataset.targets
        self.triplets = self._generate_triplets()

    def __getitem__(self, index):
        anchor, positive, negative = self.triplets[index]
        anchor_img, _ = self.mnist_dataset[anchor]
        positive_img, _ = self.mnist_dataset[int(positive)]
        negative_img, _ = self.mnist_dataset[int(negative)]
        return anchor_img, positive_img, negative_img
    
    def get_default_transform(self):
        transform = transforms.Compose(
            [
                transforms.Grayscale(
                    num_output_channels=3
                ),  # Convert to 3 channel grayscale
                transforms.Resize((32, 32)),
                transforms.ToTensor(),
            ]
        )
        return transform

    def __len__(self):
        return len(self.triplets)

    def _generate_triplets(self, debug=True):
        triplets = []
        for i in range(len(self.mnist_dataset)):
            anchor_img, anchor_label = self.mnist_dataset[i]
            positive_indices = torch.where(self.targets == anchor_label)[0]
            negative_indices = torch.where(self.targets != anchor_label)[0]

            if len(positive_indices) < 2 or len(negative_indices) < 1:
                continue

            positive_idx = positive_indices[torch.randint(high=len(positive_indices), size=(1,))]
            negative_idx = negative_indices[torch.randint(high=len(negative_indices), size=(1,))]

            triplets.append((i, positive_idx, negative_idx))

            if debug and i > 100:
                break
        return triplets
    
def triplet_collate(batch):
    # concatenate anchor, positive, and negative tensors
    anchor, positive, negative = zip(*batch)
    anchor = torch.stack(anchor, dim=0)
    positive = torch.stack(positive, dim=0)
    negative = torch.stack(negative, dim=0)

    # return triplet tensor
    return torch.stack((anchor, positive, negative), dim=0)


In [112]:
train_dataset = TripletsMNIST()

In [113]:
train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=10, shuffle=True, collate_fn=triplet_collate
    )

In [116]:
for i,(a,p,n) in enumerate(train_loader):
    # print(f"i: {i}, n: {n.shape}, a: {a.shape}")
    print(a.shape)
    if i > 10: 
        break


torch.Size([10, 3, 32, 32])
torch.Size([10, 3, 32, 32])
torch.Size([10, 3, 32, 32])
torch.Size([10, 3, 32, 32])
torch.Size([10, 3, 32, 32])
torch.Size([10, 3, 32, 32])
torch.Size([10, 3, 32, 32])
torch.Size([10, 3, 32, 32])
torch.Size([10, 3, 32, 32])
torch.Size([10, 3, 32, 32])
torch.Size([2, 3, 32, 32])
